In [ ]:
!pip install --upgrade gspread
!pip install --upgrade gspread-pandas
!pip install --upgrade gspread-dataframe

import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import Input, concatenate, Embedding, Reshape, Conv2D, MaxPooling2D
from keras.layers import Flatten, concatenate, Lambda, Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.regularizers import l2, l1_l2
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from google.colab import auth
auth.authenticate_user()
import gspread
import gspread_dataframe 
from oauth2client.client import GoogleCredentials

files = []
files.append(np.array(pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/10set (đủ mã)/AMD_Benign_ver2(tong hop)/file-0.csv", header=None, skiprows=1)))
files.append(np.array(pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/10set (đủ mã)/AMD_Benign_ver2(tong hop)/file-1.csv", header=None, skiprows=1)))
files.append(np.array(pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/10set (đủ mã)/AMD_Benign_ver2(tong hop)/file-2.csv", header=None, skiprows=1)))
files.append(np.array(pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/10set (đủ mã)/AMD_Benign_ver2(tong hop)/file-3.csv", header=None, skiprows=1)))
files.append(np.array(pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/10set (đủ mã)/AMD_Benign_ver2(tong hop)/file-4.csv", header=None, skiprows=1)))
files.append(np.array(pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/10set (đủ mã)/AMD_Benign_ver2(tong hop)/file-5.csv", header=None, skiprows=1)))
files.append(np.array(pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/10set (đủ mã)/AMD_Benign_ver2(tong hop)/file-6.csv", header=None, skiprows=1)))
files.append(np.array(pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/10set (đủ mã)/AMD_Benign_ver2(tong hop)/file-7.csv", header=None, skiprows=1)))
files.append(np.array(pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/10set (đủ mã)/AMD_Benign_ver2(tong hop)/file-8.csv", header=None, skiprows=1)))
files.append(np.array(pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/10set (đủ mã)/AMD_Benign_ver2(tong hop)/file-9.csv", header=None, skiprows=1)))

Requirement already up-to-date: gspread in /usr/local/lib/python3.6/dist-packages (3.6.0)
Requirement already up-to-date: gspread-pandas in /usr/local/lib/python3.6/dist-packages (2.2.3)
Requirement already up-to-date: gspread-dataframe in /usr/local/lib/python3.6/dist-packages (3.2.0)


In [ ]:
def tranform(data):
  deep_input_length = data.shape[1]
  size = int(math.ceil(deep_input_length**0.5))
  padding = size**2 - deep_input_length

  data = np.concatenate((data, np.zeros((data.shape[0], padding))), 1)
  data = data.reshape(data.shape[0], size, size, 1)
  return data

def WnDCNN(wide_input_shape, deep_input_shape):
  wide_input_shape = (wide_input_shape[1], )
  deep_input_shape = (deep_input_shape[1], deep_input_shape[2], deep_input_shape[3])
  #wide
  wide_input = Input(shape=wide_input_shape, name='wide_input')
  #deep
  deep_input = Input(shape=deep_input_shape, name='deep_input')
  conv1 = Conv2D(32, kernel_size=2, activation='relu', padding="same" ,input_shape=(128, 128, 1))(deep_input)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

  conv2 = Conv2D(32, kernel_size=2, activation='relu', padding="same")(pool1)
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

  conv3 = Conv2D(64, kernel_size=2, activation='relu', padding="same")(pool2)
  pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

  flatten_per = Flatten()(pool3)

  deep = Dense(1024, activation='relu')(flatten_per)
  deep = Dropout(0.3)(deep)
  deep = Dense(512, activation='relu')(deep)
  deep = Dropout(0.3)(deep)
  #wide & deep
  wide_deep_input = concatenate([wide_input, deep])
  wide_deep_out = Dense(228, activation='softmax', name='wide_deep')(wide_deep_input)
  wide_deep = Model(inputs=[wide_input, deep_input], outputs=wide_deep_out)
  wide_deep.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
  wide_deep.summary()
  return wide_deep

Bước 1: 4 máy train độc lập: val - file0, test - file1
*   server (file2)
*   client1 (file3)
*   client2 (file 4)
*   client3 (file 5)



In [ ]:
x_val_wide = files[0][:, 16386:].astype('float')
x_val_deep = tranform(files[0][:, 3:16386]).astype('float')
y_val = to_categorical(files[0][:, 2].astype('int'), 228)

x_test_wide = files[1][:, 16386:].astype('float')
x_test_deep = tranform(files[1][:, 3:16386]).astype('float')
y_test = files[1][:, 2].astype('int')

wide_input_shape = x_val_wide.shape
deep_input_shape = x_val_deep.shape
#server
print('===============  Server  ================')
x_train_wide = files[2][:, 16386:].astype('float')
x_train_deep = tranform(files[2][:, 3:16386]).astype('float')
y_train = to_categorical(files[2][:, 2].astype('int'), 228)
server = WnDCNN(wide_input_shape, deep_input_shape)
callbacks = [EarlyStopping(monitor='val_loss', patience=2), ModelCheckpoint(filepath='/content/drive/My Drive/best_model_server.h5', monitor='val_loss', save_best_only=True)]
server.fit([x_train_wide, x_train_deep], y_train, epochs=30, batch_size=64, validation_data=([x_val_wide, x_val_deep], y_val), callbacks=callbacks)
print('============== End Server ===============')
del x_train_wide, x_train_deep, y_train
#client1
print('===============  Client1  ================')
x_train_wide = files[3][:, 16386:].astype('float')
x_train_deep = tranform(files[3][:, 3:16386]).astype('float')
y_train = to_categorical(files[3][:, 2].astype('int'), 228)
client1 = WnDCNN(wide_input_shape, deep_input_shape)
callbacks = [EarlyStopping(monitor='val_loss', patience=2), ModelCheckpoint(filepath='/content/drive/My Drive/best_model_client1.h5', monitor='val_loss', save_best_only=True)]
client1.fit([x_train_wide, x_train_deep], y_train, epochs=30, batch_size=64, validation_data=([x_val_wide, x_val_deep], y_val), callbacks=callbacks)
print('============== End Client1 ===============')
del x_train_wide, x_train_deep, y_train
#client2
print('===============  Client2  ================')
x_train_wide = files[4][:, 16386:].astype('float')
x_train_deep = tranform(files[4][:, 3:16386]).astype('float')
y_train = to_categorical(files[4][:, 2].astype('int'), 228)
client2 = WnDCNN(wide_input_shape, deep_input_shape)
callbacks = [EarlyStopping(monitor='val_loss', patience=2), ModelCheckpoint(filepath='/content/drive/My Drive/best_model_client2.h5', monitor='val_loss', save_best_only=True)]
client2.fit([x_train_wide, x_train_deep], y_train, epochs=30, batch_size=64, validation_data=([x_val_wide, x_val_deep], y_val), callbacks=callbacks)
print('============== End Client2 ===============')
del x_train_wide, x_train_deep, y_train
#client3
print('===============  Client3  ================')
x_train_wide = files[5][:, 16386:].astype('float')
x_train_deep = tranform(files[5][:, 3:16386]).astype('float')
y_train = to_categorical(files[5][:, 2].astype('int'), 228)
client3 = WnDCNN(wide_input_shape, deep_input_shape)
callbacks = [EarlyStopping(monitor='val_loss', patience=2), ModelCheckpoint(filepath='/content/drive/My Drive/best_model_client3.h5', monitor='val_loss', save_best_only=True)]
client3.fit([x_train_wide, x_train_deep], y_train, epochs=30, batch_size=64, validation_data=([x_val_wide, x_val_deep], y_val), callbacks=callbacks)
print('============== End Client3 ===============')
del x_train_wide, x_train_deep, y_train

===============  Server  ================
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
deep_input (InputLayer)         [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 32) 160         deep_input[0][0]                 
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 64, 64, 32)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 32)   4128        max_pooling2d[0][0]              
____________________________________________________

In [ ]:
y_pred = server.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
print('Server: ', accuracy_score(y_test, y_pred))
y_pred = client1.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
print('Client1: ', accuracy_score(y_test, y_pred))
y_pred = client2.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
print('Client2: ', accuracy_score(y_test, y_pred))
y_pred = client3.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
print('Client3: ', accuracy_score(y_test, y_pred))

Server:  0.9206963249516441
Client1:  0.9214700193423597
Client2:  0.9137330754352031
Client3:  0.9315280464216634


Bước 2: Trung bình cộng bộ trọng số của 4

In [ ]:
weights_avg = (np.array(server.get_weights()) + np.array(client1.get_weights()) + np.array(client2.get_weights()) + np.array(client3.get_weights())) / 4
server.set_weights(weights_avg)
client1.set_weights(weights_avg)
client2.set_weights(weights_avg)
client3.set_weights(weights_avg)

y_pred = server.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
print('Server: ', accuracy_score(y_test, y_pred))
y_pred = client1.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
print('Client1: ', accuracy_score(y_test, y_pred))
y_pred = client2.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
print('Client2: ', accuracy_score(y_test, y_pred))
y_pred = client3.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
print('Client3: ', accuracy_score(y_test, y_pred))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


Server:  0.8731141199226305
Client1:  0.8731141199226305
Client2:  0.8731141199226305
Client3:  0.8731141199226305


Bước 3: Lấy 2 máy tiếp tục train


*   Client1(files[6])
*   Client2(files[7])



In [ ]:
#client1
print('===============  Client1  ================')
x_train_wide = files[6][:, 16386:].astype('float')
x_train_deep = tranform(files[6][:, 3:16386]).astype('float')
y_train = to_categorical(files[6][:, 2].astype('int'), 228)
callbacks = [EarlyStopping(monitor='val_loss', patience=2), ModelCheckpoint(filepath='/content/drive/My Drive/best_model_client1.h5', monitor='val_loss', save_best_only=True)]
client1.fit([x_train_wide, x_train_deep], y_train, epochs=30, batch_size=64, validation_data=([x_val_wide, x_val_deep], y_val), callbacks=callbacks)
print('============== End Client1 ===============')
del x_train_wide, x_train_deep, y_train
#client2
print('===============  Client2  ================')
x_train_wide = files[7][:, 16386:].astype('float')
x_train_deep = tranform(files[7][:, 3:16386]).astype('float')
y_train = to_categorical(files[7][:, 2].astype('int'), 228)
callbacks = [EarlyStopping(monitor='val_loss', patience=2), ModelCheckpoint(filepath='/content/drive/My Drive/best_model_client2.h5', monitor='val_loss', save_best_only=True)]
client2.fit([x_train_wide, x_train_deep], y_train, epochs=30, batch_size=64, validation_data=([x_val_wide, x_val_deep], y_val), callbacks=callbacks)
print('============== End Client2 ===============')
del x_train_wide, x_train_deep, y_train

===============  Client1  ================
Epoch 1/30
41/41 [==============================] - 1s 34ms/step - loss: 0.4899 - accuracy: 0.9310 - val_loss: 0.2965 - val_accuracy: 0.9602
Epoch 2/30
41/41 [==============================] - 1s 24ms/step - loss: 0.2391 - accuracy: 0.9601 - val_loss: 0.2434 - val_accuracy: 0.9590
Epoch 3/30
41/41 [==============================] - 1s 24ms/step - loss: 0.1861 - accuracy: 0.9670 - val_loss: 0.2259 - val_accuracy: 0.9598
Epoch 4/30
41/41 [==============================] - 1s 24ms/step - loss: 0.1445 - accuracy: 0.9756 - val_loss: 0.2091 - val_accuracy: 0.9652
Epoch 5/30
41/41 [==============================] - 1s 24ms/step - loss: 0.1189 - accuracy: 0.9845 - val_loss: 0.2023 - val_accuracy: 0.9663
Epoch 6/30
41/41 [==============================] - 1s 24ms/step - loss: 0.1040 - accuracy: 0.9880 - val_loss: 0.1930 - val_accuracy: 0.9667
Epoch 7/30
41/41 [==============================] - 1s 27ms/step - loss: 0.0853 - accuracy: 0.9899 - val_loss: 

In [ ]:
y_pred = server.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
print('Server: ', accuracy_score(y_test, y_pred))
y_pred = client1.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
print('Client1: ', accuracy_score(y_test, y_pred))
y_pred = client2.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
print('Client2: ', accuracy_score(y_test, y_pred))
y_pred = client3.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
print('Client3: ', accuracy_score(y_test, y_pred))

Server:  0.8731141199226305
Client1:  0.9694390715667311
Client2:  0.9647969052224371
Client3:  0.8731141199226305


Bước 4: Tính trung bình cộng của client1 và client 2 vừa train, cập nhật lại 4 máy

In [ ]:
weights_avg = (np.array(server.get_weights()) + np.array(client1.get_weights()) + np.array(client2.get_weights())) / 3
server.set_weights(weights_avg)
client1.set_weights(weights_avg)
client2.set_weights(weights_avg)
client3.set_weights(weights_avg)

y_pred = server.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
print('Server: ', accuracy_score(y_test, y_pred))
y_pred = client1.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
print('Client1: ', accuracy_score(y_test, y_pred))
y_pred = client2.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
print('Client2: ', accuracy_score(y_test, y_pred))
y_pred = client3.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
print('Client3: ', accuracy_score(y_test, y_pred))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


Server:  0.9609284332688588
Client1:  0.9609284332688588
Client2:  0.9609284332688588
Client3:  0.9609284332688588


Bước 5: Lặp lại bước 3 và 4

In [ ]:
#========================Lặp lại bước 3==============================
#client1
print('===============  Client1  ================')
x_train_wide = files[8][:, 16386:].astype('float')
x_train_deep = tranform(files[8][:, 3:16386]).astype('float')
y_train = to_categorical(files[8][:, 2].astype('int'), 228)
callbacks = [EarlyStopping(monitor='val_loss', patience=2), ModelCheckpoint(filepath='/content/drive/My Drive/best_model_client1.h5', monitor='val_loss', save_best_only=True)]
client1.fit([x_train_wide, x_train_deep], y_train, epochs=30, batch_size=64, validation_data=([x_val_wide, x_val_deep], y_val), callbacks=callbacks)
print('============== End Client1 ===============')
del x_train_wide, x_train_deep, y_train
#client2
print('===============  Client2  ================')
x_train_wide = files[9][:, 16386:].astype('float')
x_train_deep = tranform(files[9][:, 3:16386]).astype('float')
y_train = to_categorical(files[9][:, 2].astype('int'), 228)
callbacks = [EarlyStopping(monitor='val_loss', patience=2), ModelCheckpoint(filepath='/content/drive/My Drive/best_model_client2.h5', monitor='val_loss', save_best_only=True)]
client2.fit([x_train_wide, x_train_deep], y_train, epochs=30, batch_size=64, validation_data=([x_val_wide, x_val_deep], y_val), callbacks=callbacks)
print('============== End Client2 ===============')
del x_train_wide, x_train_deep, y_train
y_pred = server.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
sv_b3 = accuracy_score(y_test, y_pred)
y_pred = client1.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
c1_b3 = accuracy_score(y_test, y_pred)
y_pred = client2.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
c2_b3 = accuracy_score(y_test, y_pred)
y_pred = client3.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
c3_b3 = accuracy_score(y_test, y_pred)
#========================Lặp lại bước 4==============================
weights_avg = (np.array(server.get_weights()) + np.array(client1.get_weights()) + np.array(client2.get_weights())) / 3
server.set_weights(weights_avg)
client1.set_weights(weights_avg)
client2.set_weights(weights_avg)
client3.set_weights(weights_avg)

print('==================Kết quả lặp lại bước 3=================')
print('Server: ', sv_b3)
print('Client1: ', c1_b3)
print('Client2: ', c2_b3)
print('Client3: ', c3_b3)
print('==================Kết quả lặp lại bước 4=================')
y_pred = server.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
print('Server: ', accuracy_score(y_test, y_pred))
y_pred = client1.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
print('Client1: ', accuracy_score(y_test, y_pred))
y_pred = client2.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
print('Client2: ', accuracy_score(y_test, y_pred))
y_pred = client3.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
print('Client3: ', accuracy_score(y_test, y_pred))

===============  Client1  ================
Epoch 1/30
41/41 [==============================] - 1s 35ms/step - loss: 0.2081 - accuracy: 0.9604 - val_loss: 0.1823 - val_accuracy: 0.9675
Epoch 2/30
41/41 [==============================] - 1s 24ms/step - loss: 0.1295 - accuracy: 0.9771 - val_loss: 0.1694 - val_accuracy: 0.9779
Epoch 3/30
41/41 [==============================] - 1s 24ms/step - loss: 0.1000 - accuracy: 0.9833 - val_loss: 0.1622 - val_accuracy: 0.9737
Epoch 4/30
41/41 [==============================] - 1s 24ms/step - loss: 0.0881 - accuracy: 0.9856 - val_loss: 0.1515 - val_accuracy: 0.9764
Epoch 5/30
41/41 [==============================] - 1s 24ms/step - loss: 0.0741 - accuracy: 0.9899 - val_loss: 0.1515 - val_accuracy: 0.9749
Epoch 6/30
41/41 [==============================] - 1s 24ms/step - loss: 0.0602 - accuracy: 0.9919 - val_loss: 0.1452 - val_accuracy: 0.9756
Epoch 7/30
41/41 [==============================] - 1s 24ms/step - loss: 0.0502 - accuracy: 0.9938 - val_loss: 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


==================Kết quả lặp lại bước 3=================
Server:  0.9609284332688588
Client1:  0.9748549323017408
Client2:  0.9756286266924564
Client3:  0.9609284332688588
==================Kết quả lặp lại bước 4=================
Server:  0.9744680851063829
Client1:  0.9744680851063829
Client2:  0.9744680851063829
Client3:  0.9744680851063829


Bước 6: Chạy WnDCNN bình thường với train[file2, file9], val[file0], test[file1]

In [ ]:
# del server, client1, client2, client3, callbacks, files[0], files[1], y_pred#, y_train, y_val, x_train_deep, x_train_wide, x_val_deep, x_val_wide

train = np.concatenate([files[i] for i in range(8)], axis=0)
x_train_wide = train[:, 16386:].astype('float')
x_train_deep = tranform(train[:, 3:16386]).astype('float')
y_train = to_categorical(train[:, 2].astype('int'), 228)
model = WnDCNN(wide_input_shape, deep_input_shape)
callbacks = [EarlyStopping(monitor='val_loss', patience=2), ModelCheckpoint(filepath='/content/drive/My Drive/best_model.h5', monitor='val_loss', save_best_only=True)]
model.fit([x_train_wide, x_train_deep], y_train, epochs=30, batch_size=64, validation_data=([x_val_wide, x_val_deep], y_val), callbacks=callbacks)

y_pred = model.predict([x_test_wide, x_test_deep])
y_pred = np.argmax(y_pred, axis=-1)
print('model: ', accuracy_score(y_test, y_pred))

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
deep_input (InputLayer)         [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 128, 128, 32) 160         deep_input[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_15 (MaxPooling2D) (None, 64, 64, 32)   0           conv2d_15[0][0]                  
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 64, 64, 32)   4128        max_pooling2d_15[0][0]           
____________________________________________________________________________________________